In [1]:
import os

[ file for file in os.listdir("C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c2/") if os.path.isfile(file) and file.endswith(".tif")]

[]

In [3]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchonia_analysis

def analisis_calcio(image_path, min_dist_between_maxs, ancho_corte=5):
    image = image_data.get_image_data(image_path)
    import cv2
    # Select ROI
    fromCenter = False
    showCrosshair = False
    seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

    # Crop image
    x_start = int(seleted_parameters[1])
    x_end = x_start + int(seleted_parameters[3])
    y_start = int(seleted_parameters[0])
    y_end = y_start + int(seleted_parameters[2])

    image = image_data.crop_vertical(image, x_start, x_end)
    image = image_data.crop_horizontal(image, y_start, y_end)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte)


# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c2/2b000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

analisis_calcio(path, min_dist_between_maxs, ancho_corte)


{'image': {'intensities': array([2520, 2511, 2502, 2485, 2475, 2464, 2433, 2416, 2405, 2393, 2381,
         2355, 2335, 2315, 2308, 2292, 2279, 2268, 2261, 2255, 2261, 2290,
         2343, 2393, 2424, 2526, 2553, 2644, 2706, 2768, 2837, 2900, 2956,
         3015, 3072, 3127, 3166, 3180, 3200, 3209, 3198, 3181, 3170, 3158,
         3128, 3093, 3061, 3046, 3033, 3019, 2992, 2976, 2959, 2940, 2920,
         2900, 2870, 2851, 2832, 2824, 2814, 2802, 2775, 2747, 2725, 2701,
         2691, 2685, 2680, 2662, 2642, 2611, 2600, 2580, 2565, 2548, 2537,
         2524, 2513, 2491, 2473, 2453, 2446, 2434, 2415, 2400, 2386, 2374,
         2346, 2339, 2327, 2318, 2309, 2306, 2297, 2284, 2272, 2271, 2277,
         2292, 2330, 2391, 2456, 2522, 2572, 2647, 2702, 2776, 2844, 2908,
         2965, 3032, 3069, 3123, 3176, 3209, 3221, 3227, 3217, 3204, 3184,
         3157, 3138, 3111, 3094, 3082, 3051, 3035, 3013, 2991, 2980, 2969,
         2951, 2915, 2893, 2878, 2869, 2852, 2833, 2802, 2790, 2781, 2773,
 

In [5]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchonia_analysis

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20


# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c2/"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ image_data.get_image_data(image_path) for image_path in images_paths ]

In [6]:
import cv2

fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.waitKey(0)
cv2.destroyAllWindows()

images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]

analisis_calcio(images, min_dist_between_maxs, ancho_corte)

coso = [ dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte) for image in images ]
coso

SystemError: <built-in function imread> returned NULL without setting an error

In [47]:
coso

NameError: name 'coso' is not defined